In [1]:
## import 套件
import torch
import pandas as pd
import numpy as np
import torch.nn as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AdamW ##新ㄉ 好像比較好
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import BertModel
from transformers import BertConfig

from matplotlib import pyplot as plt

In [3]:
columns_name = ['type','title','text']
dftest = pd.read_csv('./data_after_sep/test.tsv',sep = '\t',names = columns_name)
dfdev = pd.read_csv('./data_after_sep/dev.tsv',sep = '\t',names = columns_name)

model_path = './bert_pretrain_news/'
tokenizer = BertTokenizer.from_pretrained(model_path)

In [5]:
class TestDataset(Dataset):
    def __init__(self, input_dict, y):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y
    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]
        return input_id, tokentype, attentionmask, y
    def __len__(self):
        return len(self.input_ids)

In [7]:
pred_list = np.array([])


(0,)


In [15]:
BATCH_SIZE = 2
test_x = dftest['text'].tolist()
test_input_dict = tokenizer.batch_encode_plus(test_x,
                                             add_special_tokens = True,
                                             max_length = 512,
                                             truncation = True,
                                             return_special_tokens_mask = True,
                                             pad_to_max_length = True,
                                             return_tensors = 'pt')
test_y = np.array(dftest['type'].tolist())
testset = TestDataset(test_input_dict, test_y)
testloader = DataLoader(testset, batch_size = BATCH_SIZE, shuffle = True)

In [10]:
# one model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## 類別對應 0：政治 1：生活 2：國際 3：體育 4：娛樂 5：社會 6：財經
def get_test_acc(model, testloader):
    model.eval()  ##test mode
    total = 0 ##total_num
    correct = 0 ##correct_num
    with torch.no_grad():   ##eval不計算gradient 
        for data in testloader:
            tokens_tensors , segment_tensors,masks_tensors,labels = [t.to(device) for t in data]
            outputs = model(input_ids = tokens_tensors,
                           token_type_ids = segment_tensors,
                           attention_mask = masks_tensors,
                           labels = labels)
            pred = torch.argmax(outputs[1],dim=-1)
            total += labels.size()[0]
            correct += (pred == labels).sum().item()
    
    return correct/total

In [ ]:
# three ensemble
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## 類別對應 0：政治 1：生活 2：國際 3：體育 4：娛樂 5：社會 6：財經
def get_test_ensemble_acc(model_list, testloader,batch_size):
    total = 0 ##total_num
    correct = 0 ##correct_num
    pred_list = np.array([])
    with torch.no_grad():   ##eval不計算gradient 
        for data in testloader:
            pred = torch.empty(batch_size, 7, dtype=torch.float)
            pred = pred.to(device)
            for model in model_list:
                model.eval()
                tokens_tensors , segment_tensors,masks_tensors,labels = [t.to(device) for t in data]
                outputs = model(input_ids = tokens_tensors,
                                token_type_ids = segment_tensors,
                                attention_mask = masks_tensors,
                                labels = labels)
                print(pred)
                print(outputs[1])
                pred += outputs[1]
            pred = torch.argmax(outputs[1],dim=-1)
            total += labels.size()[0]
            correct += (pred == labels).sum().item()
    
    return correct/total

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_labels = 7 
model = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
model = model.to(device)
model.load_state_dict(torch.load('/home/nlp/NewsClassify/BERT_for_xgboost_0.pkl'))

model.eval()
model1 = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
model1 = model1.to(device)
model1.load_state_dict(torch.load('/home/nlp/NewsClassify/BERT_for_xgboost_1.pkl'))
model1.eval()

model2 = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
model2 = model2.to(device)
model2.load_state_dict(torch.load('/home/nlp/NewsClassify/BERT_for_xgboost_2.pkl'))
model2.eval()
model_list = [model,model1,model2]


Some weights of the model checkpoint at ./bert_pretrain_news/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

RuntimeError: Error(s) in loading state_dict for BertForSequenceClassification:
	Missing key(s) in state_dict: "bert.embeddings.position_ids". 

In [17]:
total = 0 ##total_num
correct = 0 ##correct_num
pred_list = np.array([])
with torch.no_grad():   ##eval不計算gradient 
    for data in testloader:
        pred = torch.zeros(BATCH_SIZE, 7, dtype=torch.float)
        pred = pred.to(device)
        for model in model_list:
            model.eval()
            tokens_tensors , segment_tensors,masks_tensors,labels = [t.to(device) for t in data]
            outputs = model(input_ids = tokens_tensors,
                            token_type_ids = segment_tensors,
                            attention_mask = masks_tensors,
                            labels = labels)
            pred += outputs[1]
        pred = torch.argmax(pred,dim=-1)
        total += labels.size()[0]
        correct += (pred == labels).sum().item()
    print(correct/total)

0.8574
